In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from cadquery.occ_impl.geom import Vector
from config import Config, Shape
from plate import get_base, get_screw_positions, get_key_positions, make_plate
from case import get_basic_shape,  cut_aviator_connector_hole, get_distance_between_two_shapes,  add_microcontrollerbox, make_controller_box_top_plate, CustomisableSelector

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


In [12]:
from controller import PiPico
#rkn = [10,9,7,5,3,1]
rkn = [7,10,10,10]
config=Config(rkn, bottomFillet=2, controller=PiPico(),  edgeFillet=3)
kp = get_key_positions(config)

In [84]:
config.to_json('../configs/original.json')

In [83]:
from case import make_case


def cut_holes_in_top_plate(top):
    top_plate_width = get_distance_between_two_shapes(top.edges('|Y and >Z'))['x']
    led_num=3
    led_array_start_x = -6
    led_spacing = 7

    #cut holes for LEDs
    for i in range(0,led_num):
        top= (top.faces('>Z').workplane(centerOption="CenterOfBoundBox")
            .moveTo(led_array_start_x - i*led_spacing,(-0.5))
            .circle(2)
            .cutThruAll()
        )

    # add slot for potentiometer
    circle_dia = 9.1
    flat_width = 8.1
    encoder_hole_centre = (16, -1)
    top = (top.faces('>Z').workplane(centerOption="CenterOfBoundBox")
        .center(encoder_hole_centre[0],encoder_hole_centre[1])
        .moveTo(-0.5*flat_width, ((0.5*circle_dia)**2 - (0.5*flat_width)**2)**0.5)
        .threePointArc((0, 0.5*circle_dia), (0.5*flat_width, ((0.5*circle_dia)**2 - (0.5*flat_width)**2)**0.5))
        .vLine(-10)
        .hLine(-flat_width)
        .vLine(10)
        .wire()
        .cutThruAll()
    )
    return top
case = make_case(config,modify_controller_box=cut_holes_in_top_plate)

plate= make_plate(config)
assy = cq.Assembly()
assy.add(case)
assy.add(plate,loc=cq.Location(cq.Vector(0,0,18)) )

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.77s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [19]:
top.edges('<Y').val().Center().y


69.7

In [74]:
top.edges('<Y').val().Center().y


69.7

In [76]:
dd = (top.faces('>Z').workplane(centerOption="CenterOfBoundBox")
            .center(encoder_hole_centre[0],encoder_hole_centre[1])
._findFromPoint()).y

dd-top.edges('<Y').val().Center().y



7.0

In [73]:
(top.faces('>Z').workplane(centerOption="CenterOfBoundBox")
            .center(encoder_hole_centre[0],encoder_hole_centre[1])
._findFromPoint(True))

Vector: (0.0, 0.0, 0.0)

In [88]:
def my_func(**kwargs):
    if 'b' in kwargs.keys():
        print('found b')
    return kwargs

In [90]:
my_func(a='a',b=1)

found b


{'a': 'a', 'b': 1}